# Yoneda and Coyenda

In [54]:
:ext RankNTypes
:m Control.Monad
:m Control.Applicative
newtype Yoneda f a = Yoneda { runYoneda :: forall b. (a -> b) -> f b }

instance Functor (Yoneda f) where
  fmap f y = Yoneda $ \h -> runYoneda y (h . f)
  
instance Applicative f => Applicative (Yoneda f) where
  pure x = Yoneda $ \h -> pure $ h x
--(Yoneda yab) <*> (Yoneda ya) = Yoneda $ \bc -> bc <$> (yab id <*> ya id)
  Yoneda yab <*> Yoneda ya = Yoneda $ \h -> yab (h .) <*> ya id
  
instance Monad f => Monad (Yoneda f) where
  Yoneda ya >>= ayb = Yoneda $ \bc -> ya ayb >>= flip runYoneda bc
  
instance Alternative f => Alternative (Yoneda f) where
  empty = Yoneda $ const empty
  Yoneda a <|> Yoneda b = Yoneda $ \h -> a h <|> b h

There's a contravariant `Yoneda` and also `Coyoneda`. The definitions looks very similiar, both are GADTs, one has a covariant hom, the other has the contravariant hom.

In [37]:
:ext GADTs

data Coyoneda f a where
  Coyoneda :: (b -> a) -> f b -> Coyoneda f a
  
data Contyo f a where
  Contyo :: (a -> b) -> f b -> Contyo f a

The `a` in `Contyo` is in a contravariant position. Whereas with the newtype, the `a` was in a covariant position, since the hom is presented as part of a product instead of an argument, it's negative.

In [64]:
:m Data.Functor.Contravariant

instance Contravariant (Contyo f) where
  contramap ca (Contyo ab fb) = Contyo (ab . ca) fb

instance Functor (Coyoneda f) where
  fmap f (Coyoneda ba fb) = Coyoneda (f . ba) fb
  
instance Applicative f => Applicative (Coyoneda f) where
  pure x = Coyoneda (const x) (pure x)
  Coyoneda xab fx <*> Coyoneda xa fy = Coyoneda id $ (xab <$> fx) <*> (xa <$> fy)
  
instance Monad f => Monad (Coyoneda f) where
  Coyoneda ba fb >>= ayb = Coyoneda id (do Coyoneda x y <- ayb . ba <$> fb; x <$> y)
  
instance Alternative f => Alternative (Coyoneda f) where
  empty = Coyoneda id empty
  Coyoneda ba1 fb1 <|> Coyoneda ba2 fb2 = Coyoneda id $ ba1 <$> fb1 <|> ba2 <$> fb2